In [1]:
"""
Introduction:

Grade is an autograding framework for Python which provides more flexibility than traditional Python TestCases 
by allowing for arbitrarily complex testing conditions. Gradelang is a domain specific language for the Grade framework.
It allows graders to create question structures to evaluate student executables. 

By creating Gradelang, we hope it will make it easier for teachers and graders to evaluate student code quickly, easily,
and fairly. We also hope that the ease and flexibility of testing will create more space for creativity in assignment
design.


Implementation:


Gradelang is broken up into block structures. There are four main types of blocks:

- Setup: Statements in the setup block run before every question. Setup will most likely involve statements that
         create necessary test files or check requirements for questions.
- Teardown: Statements in the teardown block run after every question. This could include operations such as cleaning
         up created files by deleting them.
- Question: Each question block runs independently from each other question and allows the grader to specify tests
            and award values associated with those questions.
- Output: This block determines how the results of our program are outputted.



Challenges:


Examples:
"""
from gradelang.interpreter import interpret

# Basic setup tests
empty = "setup {}"
interpret(empty)

trivial_passing = "setup { assert 1 == 1; }"
interpret(trivial_passing)

trivial_failing = "setup { assert 1 == 0; }"
interpret(trivial_failing)


#Question tests

empty = "question {}"
interpret(empty)

trivial_passing = "question { assert 1 == 1; }"
interpret(trivial_passing)

trivial_failing = "question { assert 0 == 1; }"
interpret(trivial_failing)

testing_output = """
question {
    run "echo hello world";
    assert "hello world" in stdout;
}
"""
interpret(testing_output)

testing_exit_success = """
question {
    run "echo hello world";
    assert exit successful;
}
"""
interpret(testing_exit_success)

name_string = 'question "named" {}'
interpret(name_string)

name_int = 'question 1 {}'
interpret(name_int)

#Teardown tests
empty = "teardown {}"
interpret(empty)

trivial_passing = "teardown { assert 1 == 1; }"
interpret(trivial_passing)

trivial_failing = "teardown { assert 0 == 1; }"
interpret(trivial_failing)

#Output tests
empty = "output {}"
interpret(empty)

json = 'output { json; }'
interpret(json)

markdown = 'output { markdown; }'
interpret(markdown)

#Testing programs
empty ="""
setup {}
question {}
teardown {}
output {}
"""
interpret(empty)

setup_failure = """
setup { assert 1 == 0; }
question { assert 1 == 1; }
teardown {}
output {}
"""
interpret(setup_failure)

proposal = """
        setup {
        }

        teardown {
        }

        output {
        }
        
        question 1 {
            # Run the program, saving output.
            run "echo", "hello world";

            # Now let's run some checks.
            assert exit successful;

            # This checks both stdout and stderr
            assert "hello" in stdout;

            award 10;
        }

        question 2  {
            run "echo", "hello world";
            assert "goodbye" not in stdout;
            award 10;
            assert "hello" in stdout;
            assert "hello" not in stderr;
            award 10;
        }

        question 3 {
            let x be Float(minvalue=1);
            run "echo", x;

            # If we want to just look at stdout.
            assert x in stdout;
            
            String y = "fish";
            run "echo", y;
            assert "fish" in stdout;
            
            let z be String();
            run "echo", z;
            assert z in stdout;
            
            let camel be Int(min_value=6);
            run "echo", camel;
            assert camel in stdout;
            
            award 50;
        }
    """
interpret(proposal)



"""
Conclusions:

"""

Question 0: 0/0.
Question 0: 0/0.
Question 0: 0/0.
Exception thrown: AssertionError()
Traceback (most recent call last):
  File "D:\gradelang\gradelang\interpreter.py", line 45, in worker
    walk(question.body)
  File "D:\gradelang\gradelang\walk.py", line 191, in walk
    return dispatch[action](ast)
  File "D:\gradelang\gradelang\walk.py", line 104, in <lambda>
    'seq': lambda ast: (walk(ast[1]), walk(ast[2])),
  File "D:\gradelang\gradelang\walk.py", line 191, in walk
    return dispatch[action](ast)
  File "D:\gradelang\gradelang\walk.py", line 110, in <lambda>
    'assert': lambda ast: _assert(ast[1]),
  File "D:\gradelang\gradelang\walk.py", line 98, in _assert
    raise AssertionError
AssertionError

Question 0: 0/0.
Question 0: 0/0.
Question named: 0/0.
Question 1: 0/0.
Question 0: 0/0.
Question 0: 0/0.
Exception thrown: AssertionError()
Traceback (most recent call last):
  File "D:\gradelang\gradelang\interpreter.py", line 43, in worker
    walk(setup)
  File "D:\gradelang\

'\nConclusions:\n\n'